This notebook automatizes the conversion of VHDR files to EDF files. 

In [ ]:
# from google.colab import drive
# drive.mount('/content/gdrive')

# main_folder = "/content/gdrive/My Drive/epigame-folder/"


In [ ]:
# !pip install mne

import mne

In [ ]:
from datetime import datetime

def format_datetime(raw_brainvision_file):
    # Fetch the start date and time from the raw data's info dictionary
    original_date_time = str(raw_brainvision_file.info['meas_date'])

    parsed_date_time = datetime.strptime(original_date_time, "%Y-%m-%d %H:%M:%S%z")
    iso8601_date_time = parsed_date_time.strftime("%Y-%m-%dT%H:%M:%S")

    print(original_date_time, "converted to", iso8601_date_time)

    raw_brainvision_file.set_meas_date(iso8601_date_time)
    
    return raw_brainvision_file

In [ ]:
import os
import mne

def convert_vhdr_to_edf(vhdr_file, output_dir):
    """
    Converts a BrainVision VHDR file to EDF format using MNE library.

    Parameters:
    - vhdr_file (str): Path to the input VHDR file.
    - output_dir (str): Directory to save the output EDF file.
    """
    # Load data using MNE
    raw = mne.io.read_raw_brainvision(vhdr_file, preload=True)

    # Extract subject ID and run number from the filename
    file_name = os.path.basename(vhdr_file)
    subject_id = file_name.split('_')[0]
    run_number = file_name.split('_')[-2]

    # Export to EDF
    edf_file = os.path.join(output_dir, f"{subject_id}_{run_number}.edf")
    mne.export.export_raw(edf_file, raw, fmt='edf', overwrite=True)

    print(f"Converted {vhdr_file} to {edf_file}")


def parse_and_convert_data(root_dir, output_dir):
    """
    Recursively parses the input directory, finds BrainVision VHDR files,
    extracts subject ID and run number, and converts them to EDF format.

    Parameters:
    - root_dir (str): Path to the main directory containing "sub-***" subdirectories.
    - output_dir (str): Directory to save the output EDF files.
    """
    for subject_dir in os.listdir(root_dir):
        subject_path = os.path.join(root_dir, subject_dir)

        # Check if it's a "sub-***" directory
        if os.path.isdir(subject_path) and subject_dir.startswith("sub-"):
            presurgery_path = os.path.join(subject_path, "ses-presurgery", "ieeg")

            # Check if "ses-presurgery/ieeg" directory exists
            if os.path.exists(presurgery_path) and os.path.isdir(presurgery_path):
                for file in os.listdir(presurgery_path):
                    # Check if the file is a BrainVision VHDR file
                    if file.endswith(".vhdr"):
                        vhdr_file = os.path.join(presurgery_path, file)
                        convert_vhdr_to_edf(vhdr_file, output_dir)

# Replace 'your_input_directory' and 'your_output_directory' with the actual paths
input_directory = '/home/kivi/Documents/Ext-val/'
output_directory = '/home/kivi/Documents/Ext-val-edf/'

# Create output directory if it doesn't exist
os.makedirs(output_directory, exist_ok=True)

# Call the parser function
parse_and_convert_data(input_directory, output_directory)
